<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-05 18:15:04--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  6.70MB/s    in 0.8s    

2022-10-05 18:15:05 (6.70 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-05 18:15:05--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [ ]:
# ! pip install -q deepmatch

     |████████████████████████████████| 147 kB 46.3 MB/s 
     |████████████████████████████████| 2.9 MB 61.2 MB/s 


In [1]:
import pandas as pd
from ast import literal_eval
from preprocess_sku import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [90]:
from preprocess_sku import gen_data_set, gen_model_input


In [2]:
%load_ext autoreload
%autoreload 2

# Data

In [ ]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','sku_number','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['sku_number','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.sku_number.nunique())
# data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(10000, 10)
46
2175


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


### Sales data

In [76]:
data = pd.read_csv('sample_data/mik_sales_sp_10052022.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
# data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
# data['hist_sku_number'] = data['most_recent_purchased_sku'] + data['most_recent_view_sku']
data.shape

(23206, 14)

In [77]:
data['user_id'].nunique(), data['sku_number'].nunique()

(200, 8036)

In [78]:
data.columns

Index(['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip',
       'most_frequent_device_class_general', 'total_order_value',
       'total_coupon_value', 'most_recent_purchased_sku',
       'most_recent_view_sku', 'category_path'],
      dtype='object')

In [79]:
data = data[['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip','most_frequent_device_class_general','category_path']]

### Item category mapping dat

In [6]:
df_item = pd.read_csv("sample_data/mik_item_07292022.csv", index_col=False)
df_item.shape

(420317, 2)

In [13]:
df_item.head(2)

,item_id,category_path
0,88663500097650688,root//Shop Categories//Home & Decor//Wall Dcor...
1,88663500097650689,root//Shop Categories//Home & Decor//Wall Dcor...


In [64]:
data['most_recent_view_sku'].apply(lambda x: len(x)).describe()

count    14592.000000
mean         5.140831
std          7.971437
min          0.000000
25%          1.000000
50%          2.000000
75%          6.000000
max         84.000000
Name: most_recent_view_sku, dtype: float64

# Feature Engineering

In [80]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip','most_frequent_device_class_general']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [81]:
# view_item_set = list(set(data['most_recent_view_sku'].sum()))
# full_item_set = list(set(list(data['sku_number'].unique()) + view_item_set))
# len(full_item_set)

In [82]:
feature_max_idx = {}
for feature in sparse_features:
    # if feature == 'sku_number':      
    #   lbe_sku = LabelEncoder()
    #   lbe_sku.fit(full_item_set)
    #   data[feature] = lbe_sku.transform(data[feature]) + 1
    #   feature_max_idx[feature] = data[feature].max() + 1
    # else:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [83]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [84]:
data.head(2)

,data_source,user_id,sku_number,qty,trans_date,created_time,email_address,geo_zip,most_frequent_device_class_general,category_path
0,ADOBE,67,4118,1,2022-07-10,2022-07-10 09:08:18,jhitzke@yahoo.com,119,1,173
1,ADOBE,67,749,1,2022-05-27,2022-05-27 12:36:24,jhitzke@yahoo.com,119,1,667


In [85]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number"]].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((200, 2), (8036, 1))

In [86]:
# user_item_list = data.groupby("user_id")['sku_number'].apply(list)
user_item_list = data.groupby("user_id")['sku_number'].apply(list)

user_item_list.head()

user_id
1    [2738, 5785, 4318, 6007, 5783, 2337, 4455, 444...
2    [3720, 1912, 3726, 3719, 3726, 3731, 1888, 370...
3    [6729, 6001, 6730, 6727, 6728, 3726, 6730, 672...
4    [3721, 3719, 3708, 4678, 4679, 3721, 3512, 370...
5    [4397, 4965, 4233, 1654, 4241, 374, 6251, 4963...
Name: sku_number, dtype: object

In [91]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 200/200 [00:00<00:00, 234.20it/s]

7 7


In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [94]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess_sku.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((45612,), (45612,), (200,), (200,))

In [114]:
train_model_input

{'user_id': array([107,  94, 186, ..., 115,  10, 169]),
 'sku_number': array([1547, 1060, 6203, ..., 7209, 4132, 3421]),
 'hist_sku_number': array([[4675, 2323, 2321, ...,    0,    0,    0],
        [3857, 3706, 3727, ..., 5769, 5674, 3911],
        [3780, 2792, 4853, ...,    0,    0,    0],
        ...,
        [7188, 7144, 7180, ..., 7222, 6788, 6803],
        [4428, 3952, 3825, ..., 4141, 4132, 2354],
        [4192, 4091, 3340, ...,    0,    0,    0]], dtype=int32),
 'hist_category_path': array([[ 14,  12,  12, ...,   0,   0,   0],
        [326,  12,  12, ..., 202, 191, 230],
        [516, 634,  95, ...,   0,   0,   0],
        ...,
        [ 64,  64,  64, ...,  64,  64,  64],
        [220, 193, 198, ..., 193, 193, 188],
        [193, 224, 165, ...,   0,   0,   0]], dtype=int32),
 'hist_len': array([22, 50, 42, ..., 50, 50, 20]),
 'category_path': array([184,  14, 679, ...,  64, 193, 233]),
 'geo_zip': array([ 92,  38, 161, ..., 136, 100, 183]),
 'most_frequent_device_class_general'

In [115]:
train_model_input['user_id'].shape

(45612,)

# Create Embeddings

In [117]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(8031, collections.Counter, 8037)

In [118]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [119]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id', 'geo_zip', 'most_frequent_device_class_general',                        
                        'hist_sku_number',
                        'hist_category_path',
                        'hist_len'
                        ]


In [120]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [122]:
user_sparse_feature_columns = user_feature_columns[0:3]
user_sparse_feature_columns

['user_id', 'geo_zip', 'most_frequent_device_class_general']

In [123]:
user_seq_sparse_feature_columns = user_feature_columns[3:-1]
user_seq_sparse_feature_columns 

['hist_sku_number', 'hist_category_path']

In [124]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [125]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [126]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [127]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb3ed5d0>,
 'geo_zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb632d90>,
 'most_frequent_device_class_general': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb635f10>,
 'hist_sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb3ed850>,
 'hist_category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb615e10>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83eb61cc50>}

# Create User Inputs


## Create Inputs Tensor

In [139]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [140]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_zip_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'most_frequent_device_class_general_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_sku_number_1:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_category_path_1:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len_1:0' shape=(None, 1) dtype=int32>]

In [141]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [142]:
item_inputs_list

[<tf.Tensor 'sku_number_1:0' shape=(None, 1) dtype=int32>]

## Embed Inputs

In [143]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [144]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [134]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [135]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [145]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id_1/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_zip_1/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_most_frequent_device_class_general_1/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d_2/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_3/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

## Combine Embeded Input (User only)

In [146]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [147]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten_1/Reshape:0' shape=(None, 112) dtype=float32>

# Sepecify Model Layers

### User Layers

In [148]:
from tensorflow.keras import layers

In [149]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [151]:
# from deepctr.layers import DNN
from model_utils import DNN

In [152]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [153]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [154]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [155]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(8037, 32)

In [ ]:
# .eval(session=K.get_session())

### Output Layer

In [156]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

In [ ]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [ ]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [157]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [158]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

In [159]:
item_features[item_feature_name]

<tf.Tensor 'sku_number_1:0' shape=(None, 1) dtype=int32>

## Build Model

In [160]:
from tensorflow.python.keras.models import Model

In [161]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [162]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [163]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 45612 samples
Epoch 1/20
45612/45612 [==============================] - 4s 98us/sample - loss: 4.8541
Epoch 2/20
45612/45612 [==============================] - 4s 84us/sample - loss: 4.7860
Epoch 3/20
45612/45612 [==============================] - 4s 84us/sample - loss: 4.7116
Epoch 4/20
45612/45612 [==============================] - 5s 109us/sample - loss: 4.6353
Epoch 5/20
45612/45612 [==============================] - 5s 105us/sample - loss: 4.5811
Epoch 6/20
45612/45612 [==============================] - 4s 83us/sample - loss: 4.5297
Epoch 7/20
45612/45612 [==============================] - 4s 84us/sample - loss: 4.5270
Epoch 8/20
45612/45612 [==============================] - 4s 83us/sample - loss: 4.4830
Epoch 9/20
45612/45612 [==============================] - 4s 84us/sample - loss: 4.4587
Epoch 10/20
45612/45612 [==============================] - 4s 83us/sample - loss: 4.4448
Epoch 11/20
45612/45612 [==============================] - 5s 113us/sample - loss: 4.4427
Epoc

In [164]:
res = model.predict(train_model_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [165]:
res

array([[5.7535205],
       [5.4441786],
       [3.535766 ],
       ...,
       [3.3918657],
       [3.8625371],
       [4.408304 ]], dtype=float32)

# Predict: Generate Item and User Embeddings

In [167]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(200, 32)
(8036, 32)


In [168]:
prob_matrix = user_embs.dot(item_embs.T)

In [169]:
user_idx = 0
a = set([x + 1 for x in prob_matrix[user_idx].argsort()[-50:]])

In [170]:
df_train = pd.DataFrame(train_set)
b = set(df_train[df_train[0] == (user_idx+1)][1].values)

In [171]:
len(a&b)

0

In [ ]:
import numpy as np
a = np.array([ 0.11175336,  0.4548011 , -0.15772209,  0.3445765 , -0.0619677 ,
        0.06464498, -0.10026787, -0.07865412, -0.05304453,  0.19952899,
        0.14534044,  0.16223468,  0.02545189, -0.24614649,  0.02315353,
        0.09893572, -0.07596418, -0.09769203,  0.02297432,  0.1150127 ,
       -0.09194405,  0.05969758, -0.01262948, -0.04111078, -0.52638304,
       -0.01993592, -0.11702564, -0.13895226, -0.04519685, -0.30629617,
       -0.02717471, -0.03397018])
b = np.array([-0.22946945,  0.31338242,  0.17805392, -0.40977573,  0.06601848,
       -0.32935393,  0.11766144, -0.08969792, -0.01543093,  0.00668303,
       -0.1148053 ,  0.02085993,  0.3859076 , -0.07861419,  0.00837284,
       -0.01701969,  0.3163019 , -0.36242014, -0.04939312, -0.02236807,
       -0.12954137,  0.1923324 ,  0.04133848,  0.03154316, -0.00369814,
        0.15590565, -0.05733275,  0.0741621 ,  0.05379602,  0.07140778,
       -0.11147177, -0.03305427])

In [ ]:
a.dot(b)

-0.06165534518726169